In [1]:
import math
import pygame
import sys

WIDTH = 1200
HEIGHT = 800
HALF_WIDTH = WIDTH // 2
HALF_HEIGHT = HEIGHT // 2
FPS = 300
TILE = 100
MAP_SCALE = 6
MAP_TILE = TILE // MAP_SCALE
MAP_POS = (0, HEIGHT - HEIGHT // MAP_SCALE)
FPS_POS = (WIDTH - 65, 5)

FOV = math.pi / 3
HALF_FOV = FOV / 2
NUM_RAYS = 300
MAX_DEPTH = 800
DELTA_ANGLE = FOV / NUM_RAYS
DIST = NUM_RAYS / (2 * math.tan(HALF_FOV))
PROJ_COEFF = 3 * DIST * TILE
SCALE = WIDTH // NUM_RAYS
TEXTURE_WIDTH = 1200 
TEXTURE_HEIGHT = 1200
TEXTURE_SCALE = TEXTURE_WIDTH // TILE

player_pos = (HALF_WIDTH, HALF_HEIGHT)
player_angle = 0
player_speed = 2

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (220, 0, 0)
GREEN = (0, 220, 0)
BLUE = (0, 0, 255)
DARKGRAY = (40, 40, 40)
PURPLE = (120, 0, 120)
SKYBLUE = (0, 186, 255)
RED = (255, 0, 0)

pygame.init()
sc = pygame.display.set_mode((WIDTH, HEIGHT))
sc_map = pygame.Surface((WIDTH // MAP_SCALE, HEIGHT // MAP_SCALE))
clock = pygame.time.Clock()

class Player:
    def __init__(self):
        self.x, self.y = player_pos
        self.angle = player_angle

    @property
    def pos(self):
        return (self.x, self.y)

    def movement(self):
        sin_a = math.sin(self.angle)
        cos_a = math.cos(self.angle)
        keys = pygame.key.get_pressed()
        if keys[pygame.K_w]:
            if ((self.x + player_speed * cos_a)  // TILE * TILE, (self.y + player_speed * sin_a) // TILE * TILE) not in world_map:
                self.x += player_speed * cos_a
                self.y += player_speed * sin_a
        if keys[pygame.K_s]:
            if ((self.x - player_speed * cos_a)  // TILE * TILE, (self.y - player_speed * sin_a) // TILE * TILE) not in world_map:
                self.x += -player_speed * cos_a
                self.y += -player_speed * sin_a
        if keys[pygame.K_a]:
            if ((self.x + player_speed * sin_a)  // TILE * TILE, (self.y - player_speed * cos_a) // TILE * TILE) not in world_map:
                self.x += player_speed * sin_a
                self.y += -player_speed * cos_a
        if keys[pygame.K_d]:
            if ((self.x - player_speed * sin_a)  // TILE * TILE, (self.y + player_speed * cos_a) // TILE * TILE) not in world_map:
                self.x += -player_speed * sin_a
                self.y += player_speed * cos_a
        if keys[pygame.K_LEFT]:
            self.angle -= 0.02
        if keys[pygame.K_RIGHT]:
            self.angle += 0.02
        
        if keys[pygame.K_ESCAPE]:
            pygame.quit()
            sys.exit()
'''
def ray_casting(sc, player_pos, player_angle):
    cur_angle = player_angle - HALF_FOV
    xo, yo = player_pos
    for ray in range(NUM_RAYS):
        sin_a = math.sin(cur_angle)
        cos_a = math.cos(cur_angle)
        for depth in range(MAX_DEPTH):
            x = xo + depth * cos_a
            y = yo + depth * sin_a

            if (x // TILE * TILE, y // TILE * TILE) in world_map:
                depth *= math.cos(player_angle - cur_angle)
                proj_height = min(PROJ_COEFF / (depth + 0.0001), HEIGHT)
                c = 255 / (1 + depth * depth * 0.0001)
                color = (c // 2, c, c // 3)
                pygame.draw.rect(sc, color, (ray * SCALE, HALF_HEIGHT - proj_height // 2, SCALE, proj_height))
                break
        cur_angle += DELTA_ANGLE
        #pygame.draw.line(sc, GREEN, player_pos, (x, y), 2)
'''
def mapping(a, b):
    return (a // TILE) * TILE, (b // TILE) * TILE

def ray_casting(sc, player_pos, player_angle):
    ox, oy = player_pos
    xm, ym = mapping(ox, oy)
    cur_angle = player_angle - HALF_FOV
    depth_v, depth_h = 1, 1
    for ray in range(NUM_RAYS):
        sin_a = math.sin(cur_angle)
        cos_a = math.cos(cur_angle)
        sin_a = sin_a if sin_a else 0.000001
        cos_a = cos_a if cos_a else 0.000001
        
        x, dx = (xm + TILE, 1) if cos_a >= 0 else (xm, -1)
        for i in range(0, WIDTH, TILE):
            depth_v = (x - ox) / cos_a
            y = oy + depth_v * sin_a
            if mapping(x + dx, y) in world_map:
                texture_v = world_map[mapping(x + dx, y)]
                break
            x += dx * TILE
 
        y, dy = (ym + TILE, 1) if sin_a >= 0 else (ym, -1)
        for i in range(0, HEIGHT, TILE):
            depth_h = (y - oy) / sin_a
            x = ox + depth_h * cos_a
            if mapping(x, y + dy) in world_map:
                texture_h = world_map[mapping(x, y + dy)]
                break
            y += dy * TILE

        depth, offset, texture = (depth_h, int(y) % TILE, texture_h) if depth_h < depth_v else (depth_v, int(x) % TILE, texture_v)
        depth *= math.cos(player_angle - cur_angle)
        proj_height = min(int(PROJ_COEFF / depth), 2 * HEIGHT)
        c = 255 / (1 + depth * depth * 0.00002)
        color = (c, c // 2, c // 3)
        wall_column = drawing.textures[texture].subsurface(offset * TEXTURE_SCALE, 0, TEXTURE_SCALE, TEXTURE_HEIGHT)
        wall_column = pygame.transform.scale(wall_column, (SCALE, proj_height))
        sc.blit(wall_column, (ray * SCALE, HALF_HEIGHT - proj_height // 2))
        cur_angle += DELTA_ANGLE

class Drawing:
    def __init__(self, sc, sc_map):
        self.sc = sc
        self.sc_map = sc_map
        self.font = pygame.font.SysFont('Arial', 36, bold=True)
        self.textures = {'1': pygame.image.load('light_wall.jpg').convert(),
                         '2': pygame.image.load('dark_wall.jpg').convert(),
                         'S': pygame.image.load('sky.jpg').convert()}

    def background(self, angle):
        #pygame.draw.rect(sc, BLUE, (0, 0, WIDTH, HALF_HEIGHT))
        sky_offset = -5 * math.degrees(angle) % WIDTH
        self.sc.blit(self.textures['S'],(sky_offset, 0))
        self.sc.blit(self.textures['S'],(sky_offset - WIDTH, 0))
        self.sc.blit(self.textures['S'],(sky_offset + WIDTH, 0))
        pygame.draw.rect(sc, DARKGRAY, (0, HALF_HEIGHT, WIDTH, HALF_HEIGHT))
    
    def world(self, player_pos, player_angle):
        ray_casting(sc, player_pos, player_angle)
    
    def fps(self, clock):
        display_fps = str(int(clock.get_fps()))
        render = self.font.render(display_fps, 0, RED)
        self.sc.blit(render, FPS_POS)
        
    
    def mini_map(self, player):
        self.sc_map.fill(BLACK)
        map_x, map_y = player.x // MAP_SCALE, player.y // MAP_SCALE
        pygame.draw.line(self.sc_map, GREEN, (map_x, map_y), (map_x + (WIDTH // MAP_SCALE) * math.cos(player.angle),
                                                      map_y + (WIDTH // MAP_SCALE) * math.sin(player.angle)), 1)
        pygame.draw.circle(self.sc_map, RED, (map_x, map_y), 4)
        for x, y in minimap:
            pygame.draw.rect(self.sc_map, GREEN, (x, y, MAP_TILE , MAP_TILE))
        self.sc.blit(self.sc_map, MAP_POS)
        
        
text_map = [
    '111111111111',
    '1......1...1',
    '1..111...1.1',
    '1....1..11.1',
    '1..2....1..1',
    '1..2...111.1',
    '1....1.....1',
    '1111111111111'
]


world_map = {}
minimap = set()
for j, row in enumerate(text_map):
    for i, char in enumerate(row):
        if char != '.':
            world_map[i * TILE, j * TILE] = char
            minimap.add((i * MAP_TILE, j * MAP_TILE))

player = Player()
drawing = Drawing(sc, sc_map)

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
            
    player.movement()
    
    drawing.background(player.angle)
    
    drawing.world(player.pos, player.angle)
    
    drawing.fps(clock)
    
    drawing.mini_map(player)

    #pygame.draw.circle(sc, GREEN, (int(player.x), int(player.y)), 12)
    #pygame.draw.line(sc, GREEN, player.pos, (player.x + WIDTH * math.cos(player.angle),
    #                                          player.y + WIDTH * math. sin(player.angle)), 2)
    #for x,y in world_map:
    #     pygame.draw.rect(sc, DARKGRAY, (x, y, TILE, TILE), 2)

    pygame.display.flip()
    clock.tick(FPS)
    #print(clock.get_fps())

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\Public\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
